Import packages

In [1]:
import qiskit
# useful additional packages
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import numpy as np
import networkx as nx

from qiskit import Aer
from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.library import TwoLocal
from qiskit_optimization.applications import Maxcut, Tsp
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.problems import QuadraticProgram
from docplex.mp.model import Model
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.algorithms import OptimizationResult

from qiskit import Aer
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver

import warnings
warnings.filterwarnings("ignore")

### Define a class inherites from the Qiskit's Optimization Applicaiton class for solving the Optimal Transport problem

In [2]:
from qiskit_optimization.applications.optimization_application import OptimizationApplication
from typing import List, Union
class OptimalTransport(OptimizationApplication):
#class OptimalTransport():
    """Optimization Application for the optimal transport problem \
    using linear programming formulation.
    Reference:
        [1]: "Computational Optimal Transport"
        https://arxiv.org/abs/1803.00567
    """
    
    def __init__(self, C: List[int], \
                 mu: List[int], \
                 nu: List[int], \
                 p_lower_bound: int, \
                 p_upper_bound: int) -> None:
        """
        Args:
            C: A 2D numpy array for the cost function
            mu: A 1D array for the row marginal
            nu: A 1D array for the columns marginal
        """
        self._C = C
        self._mu = mu
        self._nu = nu
        self._p_lower_bound = p_lower_bound
        self._p_upper_bound = p_upper_bound
        
    def interpret(self, result: Union[OptimizationResult, np.ndarray]) -> List[int]:
        """Interpret a result as cost function in numpy array

        Args:
            result : The calculated result of the problem

        Returns:
            A numpy array of cost matrix
        """
        x = self._result_to_x(result)
        m, n = len(self._mu), len(self._nu)
        return x.reshape(n, m).T
        
    def buildA(self):
        """Create a matrix to enforce the linear programming
        constraints. See p.38 in the reference.
        
        Returns:
            A: A matrix for the constraints.
        """
        m, n = len(self._C), len(self._C[0])
        A = np.zeros((m+n, m*n), dtype = np.float64)
        for i in range(m):
            for j in range(m*n):
                if j % m == i:
                    A[i][j] = 1.
        i = m
        j = 0
        while i < m+n:
            while j < (i - m + 1) * m:
                A[i][j] = 1.
                j+=1
            i += 1
        return A

    def stackColumns(self, P):
        """Stack the columns of a matrix into a column array.
        Args:
            P: A 2D matrix.
        
        Return:
            p: A 1D numpy array with stack of columns from a matrix.
        """
        return P.T.reshape(-1, 1)
        
    def to_quadratic_program(self) -> QuadraticProgram:
        """Convert a optimal transport problem instance into a
        :class:`~qiskit_optimization.problems.QuadraticProgram`

        Returns:
            The :class:`~qiskit_optimization.problems.QuadraticProgram` created
            from the optimal transport problem instance.
        """
        mdl = Model("Optimal Transport")
        c = self.stackColumns(self._C).T[0]
        x = {i: mdl.integer_var(name=f"x_{i}", \
                                lb=self._p_lower_bound, \
                                ub=self._p_upper_bound) \
             for i in range(len(c))}
        mdl.minimize(mdl.sum(c[i]*x[i] for i in x ))
        A = self.buildA()
        marginal = np.append(self._mu, self._nu)
        for row in range(len(A)):
            mdl.add_constraint(mdl.sum(A[row][i]*x[i] for i in x)\
                               == marginal[row])
        op = from_docplex_mp(mdl)
        return op
        

### Define a OT problem

In [3]:
C = np.array([[1,2,3],[9,1,4]])
mu = np.array([3,1])
nu = np.array([1,2,1])
ot = OptimalTransport(C, mu, nu, 0, 3)
op = ot.to_quadratic_program()

In [4]:
print(op.prettyprint())

Problem name: Optimal Transport

Minimize
  x_0 + 9*x_1 + 2*x_2 + x_3 + 3*x_4 + 4*x_5

Subject to
  Linear constraints (5)
    x_0 + x_2 + x_4 == 3  'c0'
    x_1 + x_3 + x_5 == 1  'c1'
    x_0 + x_1 == 1  'c2'
    x_2 + x_3 == 2  'c3'
    x_4 + x_5 == 1  'c4'

  Integer variables (6)
    0 <= x_0 <= 3
    0 <= x_1 <= 3
    0 <= x_2 <= 3
    0 <= x_3 <= 3
    0 <= x_4 <= 3
    0 <= x_5 <= 3



### Map to the Ising problem

In [5]:
from qiskit_optimization.converters import QuadraticProgramToQubo

qp2qubo = QuadraticProgramToQubo()
qubo = qp2qubo.convert(op)
qubitOp, offset = qubo.to_ising()
print("Offset:", offset)
print("Ising Hamiltonian:")
print(str(qubitOp))

Offset: 2378.5
Ising Hamiltonian:
-214.0 * IIIIIIIIIIIZ
- 428.0 * IIIIIIIIIIZI
- 340.0 * IIIIIIIIIZII
- 680.0 * IIIIIIIIZIII
- 153.5 * IIIIIIIZIIII
- 307.0 * IIIIIIZIIIII
- 275.0 * IIIIIZIIIIII
- 550.0 * IIIIZIIIIIII
- 215.0 * IIIZIIIIIIII
- 430.0 * IIZIIIIIIIII
- 337.5 * IZIIIIIIIIII
- 675.0 * ZIIIIIIIIIII
+ 122.0 * IIIIIIIIIIZZ
+ 30.5 * IIIIIIIIIZIZ
+ 61.0 * IIIIIIIIIZZI
+ 61.0 * IIIIIIIIZIIZ
+ 122.0 * IIIIIIIIZIZI
+ 122.0 * IIIIIIIIZZII
+ 30.5 * IIIIIIIZIIIZ
+ 61.0 * IIIIIIIZIIZI
+ 61.0 * IIIIIIZIIIIZ
+ 122.0 * IIIIIIZIIIZI
+ 122.0 * IIIIIIZZIIII
+ 30.5 * IIIIIZIIIZII
+ 61.0 * IIIIIZIIZIII
+ 30.5 * IIIIIZIZIIII
+ 61.0 * IIIIIZZIIIII
+ 61.0 * IIIIZIIIIZII
+ 122.0 * IIIIZIIIZIII
+ 61.0 * IIIIZIIZIIII
+ 122.0 * IIIIZIZIIIII
+ 122.0 * IIIIZZIIIIII
+ 30.5 * IIIZIIIIIIIZ
+ 61.0 * IIIZIIIIIIZI
+ 30.5 * IIIZIIIZIIII
+ 61.0 * IIIZIIZIIIII
+ 61.0 * IIZIIIIIIIIZ
+ 122.0 * IIZIIIIIIIZI
+ 61.0 * IIZIIIIZIIII
+ 122.0 * IIZIIIZIIIII
+ 122.0 * IIZZIIIIIIII
+ 30.5 * IZIIIIIIIZII
+ 61.0 * IZIIIIIIZII

### Check the result by classical solver

In [6]:
# solving Quadratic Program using exact classical eigensolver
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())
classical_result = exact.solve(op)
print(classical_result.prettyprint())
print("Result cost function:")
print(ot.interpret(classical_result))

objective function value: 7.0
variable values: x_0=1.0, x_1=0.0, x_2=1.0, x_3=1.0, x_4=1.0, x_5=0.0
status: SUCCESS
Result cost function:
[[1. 1. 1.]
 [0. 1. 0.]]


### Solve it by quantum computer

In [7]:
algorithm_globals.random_seed = 123
seed = 321
backend = Aer.get_backend("aer_simulator_statevector")
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)

In [8]:
# QAOA
meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, quantum_instance=quantum_instance))
qaoa_result = meo.solve(op)


In [9]:
print(qaoa_result.prettyprint())
print("Result cost function:")
print(ot.interpret(qaoa_result))

objective function value: 7.0
variable values: x_0=1.0, x_1=0.0, x_2=1.0, x_3=1.0, x_4=1.0, x_5=0.0
status: SUCCESS
Result cost function:
[[1. 1. 1.]
 [0. 1. 0.]]
